In [20]:
import pandas as pd
import sqlite3

# Функция обработки расписания
def process_schedule(df):
    # Удаляем лишние строки и задаём названия столбцов
    df_cleaned = df.iloc[2:].reset_index(drop=True)
    group_names = df.iloc[1, 2:8].tolist()  # Имена групп из строки 3 (индекс строки 1 с нуля)
    columns = ['Day', 'Time'] + group_names
    df_cleaned.columns = columns
    
    # Заполняем пустые ячейки в столбце Day
    df_cleaned['Day'] = df_cleaned['Day'].fillna(method='ffill')
    
    # Объединяем строки с одинаковым временем и группой
    grouped = df_cleaned.groupby(['Day', 'Time']).agg({
        group: lambda x: ' / '.join(x.dropna().astype(str)) for group in group_names
    }).reset_index()
    
    return grouped, group_names

# Основной код
def main():
    # Путь к Excel файлу
    file_path = 'rasp.xlsx'  # Укажите путь к вашему файлу
    
    # Загружаем данные
    data = pd.ExcelFile(file_path)
    sheet_name = 'Table 1'  # Укажите нужный лист
    df = data.parse(sheet_name)
    
    # Обрабатываем данные
    processed_data, group_names = process_schedule(df)
    
    # Путь к базе данных SQLite
    db_path = 'fin_schedule.db'
    conn = sqlite3.connect(db_path)
    
    # Удаляем существующую таблицу, если она есть
    conn.execute("DROP TABLE IF EXISTS schedule;")
    
    # Создаём таблицу
    table_creation_query = """
    CREATE TABLE schedule (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        day_of_week TEXT,
        time_period TEXT,
        group_name TEXT,
        schedule TEXT
    );
    """
    conn.execute(table_creation_query)
    
    # Вставляем данные
    insert_query = """
    INSERT INTO schedule (day_of_week, time_period, group_name, schedule)
    VALUES (?, ?, ?, ?);
    """
    
    for _, row in processed_data.iterrows():
        for group_name in group_names:
            if row[group_name]:  # Пропускаем пустые группы
                conn.execute(insert_query, (row['Day'], row['Time'], group_name, row[group_name]))
    
    conn.commit()
    conn.close()
    print(f"Данные успешно сохранены в базе данных: {db_path}")

# Запуск скрипта
if __name__ == '__main__':
    main()


Данные успешно сохранены в базе данных: fin_schedule.db
